Предобработка текста.

Лемматизация, токенизация, выделение частей речи.

In [1]:
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
import nltk.corpus
import nltk.tokenize
nltk.download('punkt')
import string
import gensim
from string import punctuation
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.utils.data as data_utils
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
data1 = pd.read_csv('/content/drive/My Drive/Razmetka/description.csv',delimiter=',', header=0)
col=['audio','image','title','desc']
data1.columns=col
data=pd.concat([data1]).loc[:,['title','desc']]
data

,title,desc
0,Гитара,Гитара - струнный щипковый лютневидный инструм...
1,Флейта,Флейта - духовой музыкальный инструмент (аэроф...
2,Саксофон,Саксофон - духовой язычковый музыкальный инстр...
3,Скрипка,Скрипка - струнно-смычковый музыкальный инстру...
4,Кларнет,Кларнет - язычковый деревянный духовой музыкал...
5,Труба,Труба - европейский духовой амбушюрный инструм...


In [4]:
len(data)

6

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# Загрузка списка стоп-слов
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d404c27241bd69e5e1e08996db6e3db686ed9863f71b287cddde9bcf688bab31
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [7]:
def lemmatize(doc):
    doc = re.sub("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+", ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stop_words:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)

    if len(tokens) > 0:
        return tokens
    return []

In [8]:
data['lem'] = data['desc'].apply(lemmatize)

In [9]:
data['desc'][0]

'Гитара - струнный щипковый лютневидный инструмент.\nФорма корпуса 8-образная, плоские деки крепятся к обечайкам. Шейка с 18–19 ладами на плоском грифе, заходящем на верхнюю деку в районе 12-го лада, венчается головкой с колковым механизмом. В верхней деке, у края грифа, располагается резонаторное отверстие; в нижней её части крепится мостик с порожком. Из 6 нейлоновых или карбоновых одинарных струн 3 (реже 2) первые (верхние) не имеют оплётки, с 4-й по 6-ю обмотаны канителью. Мензура около 650 мм. Стандартный строй: e1 – h – g – d – A – E. Нотируется в скрипичном ключе октавой выше фактического звучания или в виде табулатуры. Сложилась практика называть такой инструмент классической испанской гитарой. Он же обычно подразумевается при употреблении термина «гитара» без уточнения её разновидности.'

In [10]:
data['lem']

0    [гитара, струнный, щипковый, лютневидный, инст...
1    [флейта, духов, музыкальный, инструмент, аэроф...
2    [саксофон, духов, язычковый, музыкальный, инст...
3    [скрипка, струнный, смычковый, музыкальный, ин...
4    [кларнет, язычковый, деревянный, духов, музыка...
5    [труба, европейский, духов, амбушюрный, инстру...
Name: lem, dtype: object

In [11]:
data.to_csv('/content/drive/My Drive/Razmetka/preprocessing_data/data_lem.csv')

ADJF - имя прилагательное

CONJ - conjunction - союз

NOUN - имя существительное

INFN - глагол (инфинитив)

In [12]:
data['morph'] = data['lem'].copy()
morph_word=[]# список частей речи
morph_text=[] #"Работа_NOUN"
t=[]
tt=[]
for text in data['morph']:
  for word in range(len(text)):
    p=morph.parse(text[word])[0]
    t.append((p.tag.POS))
    tt.append(p.word+'_'+(p.tag.POS if p.tag.POS else 'X'))
  morph_word.append(t)
  morph_text.append(tt)
  t=[]
  tt=[]

In [13]:
data['morph']

0    [гитара, струнный, щипковый, лютневидный, инст...
1    [флейта, духов, музыкальный, инструмент, аэроф...
2    [саксофон, духов, язычковый, музыкальный, инст...
3    [скрипка, струнный, смычковый, музыкальный, ин...
4    [кларнет, язычковый, деревянный, духов, музыка...
5    [труба, европейский, духов, амбушюрный, инстру...
Name: morph, dtype: object

In [ ]:
morph_text[:]

[['гитара_NOUN',
  'струнный_ADJF',
  'щипковый_ADJF',
  'лютневидный_ADJF',
  'инструмент_NOUN',
  'форма_NOUN',
  'корпус_NOUN',
  'образный_ADJF',
  'плоский_ADJF',
  'дек_NOUN',
  'крепиться_INFN',
  'обечайка_NOUN',
  'шейк_NOUN',
  '–_X',
  'лада_NOUN',
  'плоский_ADJF',
  'гриф_NOUN',
  'заходить_INFN',
  'верхний_ADJF',
  'дек_NOUN',
  'район_NOUN',
  'го_NOUN',
  'лада_NOUN',
  'венчаться_INFN',
  'головка_NOUN',
  'колковый_ADJF',
  'механизм_NOUN',
  'в_PREP',
  'верхний_ADJF',
  'дек_NOUN',
  'край_NOUN',
  'гриф_NOUN',
  'располагаться_INFN',
  'резонаторный_ADJF',
  'отверстие_NOUN',
  'нижний_ADJF',
  'её_ADJF',
  'часть_NOUN',
  'крепиться_INFN',
  'мостик_NOUN',
  'порожек_NOUN',
  'из_PREP',
  'нейлоновый_ADJF',
  'карбоновый_ADJF',
  'одинарный_ADJF',
  'струна_NOUN',
  'редкий_ADJF',
  'первый_ADJF',
  'верхний_ADJF',
  'иметь_INFN',
  'оплётка_NOUN',
  'й_X',
  'ю_X',
  'обмотать_INFN',
  'канитель_NOUN',
  'мензура_NOUN',
  'около_PREP',
  'мм_INTJ',
  'стандартны

In [14]:
import csv


with open('/content/drive/My Drive/Razmetka/preprocessing_data/morph_text.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Записываем каждую строку из morf_text в файл CSV
    for line in morph_text:
        writer.writerow(line)

In [15]:
data_freq = data['lem'].copy()
from collections import Counter

morph = pymorphy2.MorphAnalyzer()

# Функция для получения частей речи слова
def get_pos(word):
    return morph.parse(word)[0].tag.POS

# Преобразование слов в их части речи и подсчет их частоты
pos_freqs = []
for message in data_freq:
    pos_tags = [get_pos(word) for word in message]
    pos_counter = Counter(pos_tags)
    total_words = len(message)
    pos_freq = {pos: count/total_words for pos, count in pos_counter.items()}
    pos_freqs.append(pos_freq)

In [16]:
pos_freqs

[{'NOUN': 0.43333333333333335,
  'ADJF': 0.26666666666666666,
  'INFN': 0.12222222222222222,
  None: 0.1,
  'PREP': 0.03333333333333333,
  'INTJ': 0.011111111111111112,
  'ADVB': 0.022222222222222223,
  'NPRO': 0.011111111111111112},
 {'NOUN': 0.6091954022988506,
  'ADJF': 0.22988505747126436,
  'INFN': 0.09195402298850575,
  'ADVB': 0.011494252873563218,
  'CONJ': 0.022988505747126436,
  None: 0.022988505747126436,
  'PREP': 0.011494252873563218},
 {'NOUN': 0.48,
  'ADJF': 0.28,
  'INFN': 0.1,
  'CONJ': 0.02,
  None: 0.04,
  'PRCL': 0.01,
  'ADVB': 0.05,
  'PREP': 0.02},
 {'NOUN': 0.5375,
  'ADJF': 0.225,
  'NUMR': 0.0125,
  'INFN': 0.1375,
  'PREP': 0.05,
  'ADVB': 0.025,
  'CONJ': 0.0125},
 {'NOUN': 0.4897959183673469,
  'ADJF': 0.3469387755102041,
  'INFN': 0.12244897959183673,
  'PREP': 0.02040816326530612,
  'ADVB': 0.02040816326530612},
 {'NOUN': 0.43333333333333335,
  'ADJF': 0.35,
  'INFN': 0.13333333333333333,
  'ADVB': 0.03333333333333333,
  'PREP': 0.016666666666666666,
  N

In [17]:
df_pos_freq = pd.DataFrame(pos_freqs)
df_pos_freq.fillna(0, inplace=True)
df_pos_freq.to_csv('/content/drive/My Drive/Razmetka/preprocessing_data/pos_freqs.csv', index=False) #верный формат таблицы

In [18]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
# Определение ключевых слов с использованием TF-IDF
# Преобразование лемматизированных данных в строки
data_lem_str = data['lem'].apply(lambda x: ' '.join(x))
# Вычисление TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_lem_str)
feature_names = vectorizer.get_feature_names_out()

# Функция для извлечения ключевых слов
def get_top_keywords(tfidf_matrix, feature_names, top_n=5):
    top_keywords = []
    for row in tfidf_matrix:
        sorted_nzs = np.argsort(row.data)[::-1]
        top_keywords.append([feature_names[row.indices[i]] for i in sorted_nzs[:top_n]])
    return top_keywords

top_keywords = get_top_keywords(tfidf_matrix, feature_names)

# Добавление ключевых слов к данным
data['top_keywords'] = top_keywords

# Сохранение результатов
data.to_csv('/content/drive/My Drive/Razmetka/preprocessing_data/data_with_keywords.csv', index=False)

# Вывод ключевых слов
print(data[['title', 'desc', 'top_keywords']].head())

      title                                               desc  \
0    Гитара  Гитара - струнный щипковый лютневидный инструм...   
1    Флейта  Флейта - духовой музыкальный инструмент (аэроф...   
2  Саксофон  Саксофон - духовой язычковый музыкальный инстр...   
3   Скрипка  Скрипка - струнно-смычковый музыкальный инстру...   
4   Кларнет  Кларнет - язычковый деревянный духовой музыкал...   

                                        top_keywords  
0                [гитара, дек, верхний, её, плоский]  
1   [флейта, инструмент, продольный, свистковый, др]  
2  [саксофон, разновидность, трость, одинарный, к...  
3  [музыка, смотреть, скрипка, струнный, классиче...  
4     [широкий, кларнет, музыка, духов, музыкальный]  


In [20]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [21]:
import spacy

# Визуализация зависимостей
from spacy import displacy
# Загрузка модели языка
nlp = spacy.load("ru_core_news_sm")

In [ ]:
# Функция для анализа зависимостей
def analyze_dependencies(text):
    doc = nlp(text)
    dependencies = []
    for token in doc:
        dependencies.append((token.text, token.dep_, token.head.text))
    #для визуализации зависимостей
    displacy.serve(doc, style="dep")
    return dependencies

# Применение функции анализа зависимостей к описаниям инструментов
data['dependencies'] = data['desc'].apply(analyze_dependencies)




# Сохранение результатов с зависимостями
data.to_csv('/content/drive/My Drive/Razmetka/preprocessing_data/data_with_dependencies.csv', index=False)

# Вывод данных с зависимостями
print(data[['title', 'desc', 'dependencies']].head())

/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

